In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import tqdm
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

In [3]:
def get_data():
    file = open('./data/shashlik_61_pulses.txt', 'r')
    data = file.readlines()
    data = np.array([list(map(float, experiment.split())) for experiment in data])
   
    X = data[:, 2:]
    y_baseline = data[:, 1]
    y = data[:, 0]
    
    return X, y

def get_freq_data(X, freq=20, start_point=384):
    X_freq = np.concatenate([X[:, start_point::-freq], X[:, start_point::freq]], axis=1)
    return X_freq

def signal_cyclic_shift(signal, tau):
    signal_start = signal[:-tau]
    
    new_signal = np.concatenate([signal[-tau:], signal_start])
    
    return new_signal

In [4]:
def generate_multi_signal(X_origin, tau, alpha, to_plot=False):
    first_idx, second_idx = np.random.choice(X_origin.shape[0], 2, replace=False)
    first_impulse = X_origin[first_idx]
    second_impulse = X_origin[second_idx]
    
    second_impulse = signal_cyclic_shift(second_impulse, tau)
    
    multi_impulse = first_impulse + second_impulse*alpha
    multi_impulse /= -np.min(multi_impulse)
    
    first_impulse_shifted = signal_cyclic_shift(first_impulse, mean_argmin - np.argmin(first_impulse))
    second_impulse_shifted = signal_cyclic_shift(second_impulse, mean_argmin - np.argmin(second_impulse))
    multi_impulse_shifted = signal_cyclic_shift(multi_impulse, mean_argmin - np.argmin(multi_impulse))

    if to_plot:
        plt.plot(first_impulse_shifted)
        plt.plot(second_impulse_shifted)
        plt.plot(multi_impulse_shifted)
        plt.legend(['First signal', 'Second signal', 'Sum of signals'])
        plt.show()
        
    return {'first': first_impulse_shifted,\
            'second': second_impulse_shifted,\
            'multi': multi_impulse_shifted}

In [5]:
from sklearn.utils import shuffle

def prepare_data(X_origin, tau, alpha, freq, data_size=3000, to_print=False):
    pos_size = int(data_size/2)
    neg_size = data_size - pos_size
    
    X_positive = np.array([generate_multi_signal(X_origin, tau, alpha)['first'] for i in range(pos_size)])
    y_positive = np.ones(pos_size)
    
    X_negative = np.array([generate_multi_signal(X_originm tau, alpha)['multi'] for i in range(neg_size)])
    y_negative = np.zeros(neg_size)
    
    X = np.concatenate([X_positive, X_negative])
    y = np.concatenate([y_positive, y_negative])
   
    if to_print:
        print("X positive shape:", X_positive.shape)
        print("y positive shape:", y_positive.shape)
        print("X negative shape:", X_negative.shape)
        print("y negative shape:", y_negative.shape)
        print("X shape:", X.shape)
        print("y shape:", y.shape)
    
    X, y = shuffle(X, y)

    return X, y

In [ ]:
import warnings
from sklearn.preprocessing import PolynomialFeatures
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

def get_pipelines(hyperparams = {'svc_c': 0.1, 'mlp_max_iter': 1000, 'kn_n': 5, 'pa_c': 0.0001}):
    SVC_pipeline = Pipeline([
                    ('clf', LinearSVC(C=hyperparams['svc_c'])),
                ])


    MLP_pipeline = Pipeline([
                    ('clf', MLPClassifier(max_iter=hyperparams['mlp_max_iter'])),
                ])

    KN_pipeline = Pipeline([
                    ('clf', KNeighborsClassifier(n_neighbors=hyperparams['kn_n'])),
                ])
    
    PA_pipeline = Pipeline([
                ('clf', PassiveAggressiveClassifier(C = hyperparams['pa_c'], max_iter=5000, tol=1e-3)),
            ])

    pipelines = {'svc': SVC_pipeline, 'kn': KN_pipeline, 'pa': PA_pipeline,  'mlp': MLP_pipeline}
    
    return pipelines

In [6]:
def calculate_scores(alpha_values, tau_values):
    scores_dict = {}
    
    for tau in tqdm.tqdm(tau_values):
        scores_dict[tau] = dict(zip(alpha_values, np.zeros(len(alpha_values))))
        for alpha in alpha_values:
            X, y = prepare_data(tau, alpha, data_size=2000)
            pipelines = get_pipelines()

            scoring = {'accuracy' : make_scorer(accuracy_score)}
            scores = cross_validate(pipelines['kn'], X, y, scoring=scoring, cv=5)

            scores_dict[tau][alpha] = scores["test_accuracy"].mean()
    return scores_dict

In [8]:
import pickle

def save_obj(obj, name ):
    with open('./data/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def load_obj(name ):
    with open('./data/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [9]:
def dict_to_arrays(scores_dict):
    x, y, z = [], [], []
    for tau, alpha_dict in scores_dict.items():
        for alpha, score in alpha_dict.items():
            x.append(tau)
            y.append(alpha)
            z.append(score)
            
    return [x, y, z]

In [10]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
matplotlib.use('Qt4Agg')

def plot_accuracy_3d(scores_dict):
    x, y, z = dict_to_arrays(scores_dict)
    alpha_values = list(scores_dict[list(scores_dict.keys())[0]].keys())
    
    fig1 = matplotlib.pyplot.figure(figsize=(7, 5))
    ax1 = Axes3D(fig1)

    ax1.set_yticks(np.log10(alpha_values[::10]))
    ax1.set_yticklabels(alpha_values[::10])

    ax1.set_xlabel(r"${\tau}$", fontsize=15)
    ax1.set_ylabel(r"${\alpha}$", fontsize=15)
    ax1.set_zlabel('Accuracy', fontsize=15)

    ax1.view_init(30,150)
    ax1.plot(10*x, np.log10(10*y), 10*z, 'ro', color='b', linewidth=3)

    # for angle in range(0, 220):
    #     ax1.view_init(30, angle)
    #     plt.draw()
    #     plt.pause(.001)

    plt.show()

In [11]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

def plot_color_map(scores_dict, alpha_values, tau_values, alpha_freq=10, tau_freq=4):
    _, _, z = dict_to_arrays(scores_dict)
    z = np.array(z)
    Z = z.reshape((len(tau_values), len(alpha_values)))

    plt.figure(figsize=(5, 5))
    ax = plt.gca()


    plt.xlabel("Two signals ratio", fontsize=12)
    plt.ylabel("Delta t [ns]", fontsize=12)
    plt.xticks([alpha_freq*i for i in range(len(alpha_values[::alpha_freq]))], alpha_values[::alpha_freq])
    plt.yticks([tau_freq*i for i in range(len(tau_values[::tau_freq]))], [0.2*el for el in tau_values[::tau_freq]])

    im = ax.imshow(Z, interpolation='bilinear', cmap='spring', aspect='auto')

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.5)

    plt.colorbar(im, cax=cax)
    plt.show()

In [ ]:
alpha_values = np.array([np.around(10**i, decimals=4) for i in np.arange(-3, 3.1, 0.1)])
tau_values = range(-100, 110, 10)
pipelines = get_pipelines()

for freq in range(2, 40, 2):
    X_origin, y_origin = get_data()

    X_origin = np.array([experiment - np.max(experiment) for experiment in X_origin])
    X_origin = np.array([experiment/-np.min(experiment) for experiment in X_origin])
    X_origin = get_freq_data(X_origin)
    
    y_origin = np.round(y_origin)
    y_origin = y_origin.astype(int)
    
    pipelines

    for model_name, model in pipelines.items():
        print("Model name:", model_name)

        scoring = {'accuracy' : make_scorer(accuracy_score), 
               'precision' : make_scorer(precision_score),
               'recall' : make_scorer(recall_score), 
               'f1_score' : make_scorer(f1_score)}

        scores = cross_validate(model, X, y, scoring=scoring, cv=10)
        for score in scoring.keys():
            print(score, "%0.5f (+/- %0.5f)" % (scores["test_" + score].mean(), scores["test_" + score].std() * 2))
        print('------------------------------')